In [2]:
rand(1:8)

7

In [4]:
x = [(i, rand(1:8)) for i in 1:8]

8-element Vector{Tuple{Int64, Int64}}:
 (1, 4)
 (2, 2)
 (3, 3)
 (4, 7)
 (5, 6)
 (6, 7)
 (7, 6)
 (8, 2)

In [5]:
[ i*j for (i,j) in x]

8-element Vector{Int64}:
  4
  4
  9
 28
 30
 42
 42
 16

In [8]:
a = [1, 3, 5, 7, 9, 3, 5, 7]
findall(x->x==3, a)

2-element Vector{Int64}:
 2
 6

In [58]:
using Flux

In [19]:
test_batch = [ (rand(Float32,2), rand(Float32,1)) for i in 1:3]

3-element Vector{Tuple{Vector{Float32}, Vector{Float32}}}:
 ([0.523947, 0.3047166], [0.21535373])
 ([0.13237017, 0.8566941], [0.06494528])
 ([0.9243327, 0.81498635], [0.3492868])

In [59]:
test_imag = rand(Float32,2, 100)
test_target = rand(Float32,1, 100)

1×100 Matrix{Float32}:
 0.320748  0.94637  0.392362  0.88544  …  0.296158  0.5328  0.00955468

In [60]:
model = Chain(Dense(2,3, tanh), BatchNorm(3), Dense(3,1))

Chain(
  Dense(2 => 3, tanh),                  # 9 parameters
  BatchNorm(3),                         # 6 parameters, plus 6
  Dense(3 => 1),                        # 4 parameters
)         # Total: 6 trainable arrays, 19 parameters,
          # plus 2 non-trainable, 6 parameters, summarysize 588 bytes.

In [61]:
opt = ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [35]:
model(rand(Int, 2,100))

1×100 Matrix{Float32}:
 0.668815  1.67469  0.668815  -0.668815  …  0.668815  -0.668815  0.668815

In [62]:
loss(x, y) =  sum([(y[i] .- model(x)[i]).^2 for i=1:length(y) ])/length(y)

loss (generic function with 2 methods)

In [64]:
Flux.train!(loss, Flux.params(model), [(test_imag, test_target)], opt)

In [1]:
using SymPy

In [2]:
x = symbols("x")

x

In [18]:
ex1 = sin(x)

sin(x)

In [20]:
M1 = [1.0 0.0; 0.0 1.0]
M2 = [0.0 1.0; 1.0 0.0]

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [21]:
ex2 = M1 + M2*ex1

2×2 Matrix{Sym}:
 1.00000000000000        1.0*sin(x)
       1.0*sin(x)  1.00000000000000

In [24]:
ex3 = ex2.integrate((x, 0.0, x))

2×2 Matrix{Sym}:
            1.0*x  1.0 - cos(x)
 1.0 - cos(x)             1.0*x

In [8]:
t = collect(1:10)

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [17]:
@time A = [N(ex1(i)) for i in t]

  0.039646 seconds (37.73 k allocations: 2.323 MiB, 74.94% compilation time)


10-element Vector{Int64}:
   4
   9
  16
  25
  36
  49
  64
  81
 100
 121

In [15]:
typeof(A)

Vector{Sym} (alias for Array{Sym, 1})

In [25]:
using LinearAlgebra
using Flux
using ParameterSchedulers
using SymPy


struct Env
    max_turn::Int
    num_player::Int
    val_num::Int
    br_num::Int
    fn_num::Int
    act_ind::Int
    input_dim::Int
    middle_dim::Int
    output::Int

    #training parameter
    training_step::Int
    checkpoint_interval::Int
    batch_size::Int
    η::Float32
    momentum::Float32
    scheduler

    num_simulation::Int
    α::Float32
    frac::Float32

    t_step::Int
    HS_size::Int
    Ω::Float32
    ξ::Float32
    Jz::Float32
    Jx::Float32
    hz::Float32
    H_0::Hermitian{ComplexF32, Matrix{ComplexF32}}
    V_t::Hermitian{ComplexF32, Matrix{ComplexF32}}
    dt::Float32

    Cb::Int
    Ci::Float32
    C::Float32 #L2 norm weight
end

x = symbols("x")
sx = sin(x)

sin(x)

In [41]:
using ParameterSchedulers
#max_turn, num_player, num_simulation, α, frac, t_step, HS_size, Ω, ξ, Jz, Jx, hz, Cb, Ci, C
function init_Env(args::Vector{String})
    max_turn = parse(Int, args[1])
    num_player = parse(Int, args[2])
    val_num::Int = 2
    br_num::Int = 3
    fn_num::Int = 2
    act_ind = val_num+br_num+fn_num
    input_dim = act_ind*max_turn
    middle_dim = 128
    output =  act_ind + 1

    #training parameter
    training_step = 100000
    checkpoint_interval = 1000
    batch_size = 1024
    η = 1f-4
    momentum = 0.9
    scheduler = Step(λ = 2f-1, γ = Float32(0.1), step_sizes = 20000)


    num_simulation = parse(Int, args[3])
    α = parse(Float32, args[4])
    frac = parse(Float32, args[5])

    t_step = parse(Int, args[6])
    HS_size = parse(Int, args[7])
    Ω = parse(Float32, args[8])
    ξ = parse(Float32, args[9])
    Jz = parse(Float32, args[10])
    Jx = parse(Float32, args[11])
    hz = parse(Float32, args[12])
    H_0 = Hermitian([ -Jz-2hz 0 0 -Jx; 0 Jz -Jx 0; 0 -Jx Jz 0; -Jx 0 0 -Jz+2hz])
    V_t = Hermitian([ 0 -ξ -ξ 0; -ξ 0 0 -ξ; -ξ 0 0 -ξ; 0 -ξ -ξ 0])
    dt = 2pi/t_step/Ω

    Cb = parse(Int, args[13])
    Ci = parse(Float32, args[14])
    C = parse(Float32, args[15])

    return Env(max_turn, num_player, val_num, br_num, fn_num, act_ind, input_dim, middle_dim, output, training_step, checkpoint_interval, batch_size, η, momentum, scheduler, num_simulation, α, frac, t_step, HS_size, Ω, ξ, Jz, Jx, hz, H_0, V_t, dt, Cb, Ci, C)
end

init_Env (generic function with 1 method)

In [42]:
en = init_Env(["10", "2", "100", "0.3", "0.25", "100", "4", "10.0", "0.2", "1.0", "0.7", "0.5", "100", "0.1", "0.1"])

Env(10, 2, 2, 3, 2, 7, 70, 128, 8, 100000, 1000, 1024, 0.0001f0, 0.9f0, Step{Float32, Base.Iterators.Repeated{Int64}}(0.2f0, 0.1f0, Base.Iterators.Repeated{Int64}(20000)), 100, 0.3f0, 0.25f0, 100, 4, 10.0f0, 0.2f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.2f0 + 0.0f0im -0.2f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.2f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.2f0 + 0.0f0im; -0.2f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.2f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.2f0 - 0.0f0im -0.2f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.0062831854f0, 100, 0.1f0, 0.1f0)

In [105]:
function calc_Kt(history::Vector{Int}, env::Env)
    MV = []
    his = copy(history)
    #println(length(his))
    for it in 1:length(his)
        sw = pop!(his)
        if(sw==1)
            push!(MV, env.H_0)
        elseif(sw==2)
            push!(MV, env.V_t*sin(x))
        elseif(sw==3)
            A = pop!(MV)
            B = pop!(MV)
            C = A + B
            push!(MV, C)
        elseif(sw==4)
            A = pop!(MV)
            B = pop!(MV)
            C = -1.0im*(A*B - B*A)
            push!(MV, C)
        elseif(sw==5)
            A = pop!(MV)
            B = pop!(MV)
            C = (A*B + B*A)/2
            push!(MV, C)
        elseif(sw==6)
            A = pop!(MV)
            B = A.integrate((x, 0, x))
            push!(MV, B)
        end
        #@show MV
    end
    t = collect(0:env.dt:2pi)

    Ks = MV[end]
    #println(Ks)
    Kt::Vector{Hermitian{ComplexF32, Matrix{ComplexF32}}} = [Hermitian(N(Ks.subs(x,t[i]))) for i in 1:env.t_step]
    #Kt = [Hermitian(N(Ks.subs(x,t[i]))) for i in 1:env.t_step]
    return Kt
end

calc_Kt (generic function with 1 method)

In [106]:
Kt = calc_Kt([3, 6, 2, 1], en)

100-element Vector{Hermitian{ComplexF32, Matrix{ComplexF32}}}:
 [-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im]
 [-2.0f0 + 0.0f0im -3.947829f-6 + 0.0f0im -3.947829f-6 + 0.0f0im -0.7f0 + 0.0f0im; -3.947829f-6 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im -3.947829f-6 + 0.0f0im; -3.947829f-6 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im -3.947829f-6 + 0.0f0im; -0.7f0 - 0.0f0im -3.947829f-6 - 0.0f0im -3.947829f-6 - 0.0f0im 0.0f0 + 0.0f0im]
 [-2.0f0 + 0.0f0im -1.579116f-5 + 0.0f0im -1.579116f-5 + 0.0f0im -0.7f0 + 0.0f0im; -1.579116f-5 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im -1.579116f-5 + 0.0f0im; -1.579116f-5 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im -1.579116f-5 + 0.0f0im; -0.7f0 - 0.0f0im -1.579116f-5 - 0.0f0im -1.579116f-5 - 0.0f0im 0.0f0 + 0.0f0im]
 [-2.0f0 + 0.0f0im -3.55

In [72]:
test_his = [3, 6, 2, 1]
MV0 = []

Any[]

In [73]:
#A = pop!(test_his)
push!(MV0, en.H_0)
MV0[1]

4×4 Hermitian{ComplexF32, Matrix{ComplexF32}}:
 -2.0+0.0im   0.0+0.0im   0.0+0.0im  -0.7+0.0im
  0.0-0.0im   1.0+0.0im  -0.7+0.0im   0.0+0.0im
  0.0-0.0im  -0.7-0.0im   1.0+0.0im   0.0+0.0im
 -0.7-0.0im   0.0-0.0im   0.0-0.0im   0.0+0.0im

In [75]:
push!(MV0, en.V_t*sx)
@show MV0

MV0 = Any[ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], Sym[0 -0.200000002980232*sin(x) -0.200000002980232*sin(x) 0; -0.200000002980232*sin(x) 0 0 -0.200000002980232*sin(x); -0.200000002980232*sin(x) 0 0 -0.200000002980232*sin(x); 0 -0.200000002980232*sin(x) -0.200000002980232*sin(x) 0]]


2-element Vector{Any}:
 ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im]
 Sym[0 -0.200000002980232*sin(x) -0.200000002980232*sin(x) 0; -0.200000002980232*sin(x) 0 0 -0.200000002980232*sin(x); -0.200000002980232*sin(x) 0 0 -0.200000002980232*sin(x); 0 -0.200000002980232*sin(x) -0.200000002980232*sin(x) 0]

In [77]:
A = pop!(MV0)
B = A.integrate((x, 0, x))
push!(MV0, B)
@show MV0

MV0 = Any[ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], Sym[0 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0; 0.200000002980232*cos(x) - 0.200000002980232 0 0 0.200000002980232*cos(x) - 0.200000002980232; 0.200000002980232*cos(x) - 0.200000002980232 0 0 0.200000002980232*cos(x) - 0.200000002980232; 0 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0]]


2-element Vector{Any}:
 ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im]
 Sym[0 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0; 0.200000002980232*cos(x) - 0.200000002980232 0 0 0.200000002980232*cos(x) - 0.200000002980232; 0.200000002980232*cos(x) - 0.200000002980232 0 0 0.200000002980232*cos(x) - 0.200000002980232; 0 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0]

In [78]:
A = pop!(MV0)
B = pop!(MV0)
C = A+B
push!(MV0, C)
@show MV0

MV0 = Any[Sym[-2.00000000000000 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 -0.699999988079071; 0.200000002980232*cos(x) - 0.200000002980232 1.00000000000000 -0.699999988079071 0.200000002980232*cos(x) - 0.200000002980232; 0.200000002980232*cos(x) - 0.200000002980232 -0.699999988079071 1.00000000000000 0.200000002980232*cos(x) - 0.200000002980232; -0.699999988079071 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0]]


1-element Vector{Any}:
 Sym[-2.00000000000000 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 -0.699999988079071; 0.200000002980232*cos(x) - 0.200000002980232 1.00000000000000 -0.699999988079071 0.200000002980232*cos(x) - 0.200000002980232; 0.200000002980232*cos(x) - 0.200000002980232 -0.699999988079071 1.00000000000000 0.200000002980232*cos(x) - 0.200000002980232; -0.699999988079071 0.200000002980232*cos(x) - 0.200000002980232 0.200000002980232*cos(x) - 0.200000002980232 0]

In [79]:
M = MV0[end]

4×4 Matrix{Sym}:
                            -2.00000000000000  …                            -0.699999988079071
 0.200000002980232*cos(x) - 0.200000002980232     0.200000002980232*cos(x) - 0.200000002980232
 0.200000002980232*cos(x) - 0.200000002980232     0.200000002980232*cos(x) - 0.200000002980232
                           -0.699999988079071                                                0

In [80]:
t = collect(0:en.dt:2pi)

1000-element Vector{Float64}:
 0.0
 0.0062831854447722435
 0.012566370889544487
 0.01884955633431673
 0.025132741779088974
 0.03141592722386122
 0.03769911266863346
 0.043982298113405704
 0.05026548355817795
 0.05654866900295019
 ⋮
 6.226636775769293
 6.2329199612140656
 6.239203146658838
 6.24548633210361
 6.251769517548382
 6.2580527029931545
 6.264335888437927
 6.270619073882699
 6.276902259327471

In [89]:
[Hermitian(N(M.subs(x, t[i]))) for i in 1:en.t_step]

100-element Vector{Hermitian{Real, Matrix{Real}}}:
 [-2.0 0 0 -0.699999988079071; 0 1.0 -0.699999988079071 0; 0 -0.699999988079071 1.0 0; -0.699999988079071 0 0 0]
 [-2.0 -3.947829004313785e-6 -3.947829004313785e-6 -0.699999988079071; -3.947829004313785e-6 1.0 -0.699999988079071 -3.947829004313785e-6; -3.947829004313785e-6 -0.699999988079071 1.0 -3.947829004313785e-6; -0.699999988079071 -3.947829004313785e-6 -3.947829004313785e-6 0]
 [-2.0 -1.57911601636751e-5 -1.57911601636751e-5 -0.699999988079071; -1.57911601636751e-5 1.0 -0.699999988079071 -1.57911601636751e-5; -1.57911601636751e-5 -0.699999988079071 1.0 -1.57911601636751e-5; -0.699999988079071 -1.57911601636751e-5 -1.57911601636751e-5 0]
 [-2.0 -3.552952592367209e-5 -3.552952592367209e-5 -0.699999988079071; -3.552952592367209e-5 1.0 -0.699999988079071 -3.552952592367209e-5; -3.552952592367209e-5 -0.699999988079071 1.0 -3.552952592367209e-5; -0.699999988079071 -3.552952592367209e-5 -3.552952592367209e-5 0]
 [-2.0 -6.316214704737977